In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
from vega_datasets import data
import numpy as np

%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

In [2]:
table1=pd.read_csv('rankcat v2.csv').dropna()

In [3]:
table2=pd.read_csv('typeEngage.csv')
table2=table2.rename(columns={'share_on_facebook':'share '})

In [4]:
table3=pd.read_csv('rankcat and month v2.csv')

In [5]:
table4=pd.read_csv('rankbychannel.csv')
table5=pd.read_csv('rankbymonthandchannel.csv')

In [16]:
#dropdownMonth
TTable1=table4.loc[lambda table4: table4['rankengage']<21,:]

channel=['instagram', 'facebook', 'youtube', 'twitter']
month=['January','February','March','April','May','June','July','August','September','October','November','December']

dropdown = alt.binding_select(options=channel)
catselect = alt.selection_single(fields=['channel'], bind=dropdown, name="Select",init={'channel':'facebook'})
nameselect = alt.selection_single(empty='none', fields=['account_name'],init={'account_name':'อีจัน'})
highlight = alt.selection(type='single', on='mouseover',fields=['account_name'], nearest=True)

xscale=alt.Scale(type='log',domain=[1,2000000000])
yscale=alt.Scale(domain=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
# y2scales=alt.Scale(rangeStep=10)

pointsT = alt.Chart(TTable1).mark_circle(size=100).encode(
    color=alt.condition(nameselect,alt.value('white'),alt.value('gray')),
    y=alt.Y('rankengage:O',scale=yscale,title=None),
    tooltip=[alt.Tooltip('engagement',format=',.4r')]
).properties(height=500)

textT = pointsT.mark_text(
    align='left',
    baseline='middle',
    dx=7,
    fontSize=12
).encode(
    text='account_name',
    color=alt.condition(~nameselect,alt.value('gray'),alt.value('white')),
    size=alt.condition(nameselect,alt.value(18),alt.value(15))
)

Base=(pointsT+textT)

filter_dropdown = Base.add_selection(
    catselect
).transform_filter(
    catselect
).add_selection(
    nameselect
).properties(title="Top 20 Social Media Accounts")


TTable3=table5.loc[lambda table5: table5['rankengage']<11,:]
table1Base=alt.Chart(TTable3).encode(
    alt.X('month:O',sort=month,title=None),
    alt.Y('rankengage:O',title=None),
    alt.Color('account_name:N',legend=None),
    tooltip=['account_name',alt.Tooltip('engagement',format=',.4r')]
)

table1Point = table1Base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
)

table1Lines =table1Base.mark_line().encode(
     size=alt.condition(~nameselect, alt.value(1), alt.value(5)),
    color=alt.condition(nameselect,'account_name:N', alt.value('lightgray'))
).add_selection(
    nameselect
)

table1Circle=table1Base.mark_circle().encode(
     size=alt.condition(~nameselect,alt.value(80), alt.value(200)),
    color=alt.condition(nameselect,'account_name:N', alt.value('lightgray'))
)

table1Chart=table1Point+table1Lines+table1Circle
Table3=table1Chart.add_selection(
    catselect
).transform_filter(
    catselect
).properties(width=800,height=200,title='Monthly Rankings')


FBTable2=table2.loc[lambda table2: table2['channel']=='facebook',:]
FBtable2=alt.Chart(FBTable2).transform_fold(
    ['like','love','wow','haha','sad','angry','comment','share'],as_=['Type','#Engagement']
).mark_bar(color='#4267B2').encode(
    alt.X('#Engagement:Q',axis=None,scale=xscale),
    alt.Y('Type:N',title=None,axis=alt.Axis(ticks=False, domain=False)),
    alt.Tooltip('engagement:Q')
).properties(width=350,height=180,title='Facebook')

FBtext = FBtable2.mark_text(
    align='left',
    baseline='middle',
    dx=3, 
    color='white'
).encode(
    alt.Text('#Engagement:Q',format=',.4r')
)
FBchart = (FBtable2+FBtext).transform_filter(
    nameselect
)


IGTable2=table2.loc[lambda table2: table2['channel']=='instagram',:]
IGtable2=alt.Chart(IGTable2).transform_fold(
    ['like','comment','share '],as_=['Type','#Engagement']
).mark_bar(size=20,color='#8a3ab9').encode(
    alt.X('#Engagement:Q',axis=None,scale=xscale),
    alt.Y('Type:N',title=None),
    alt.Tooltip('engagement:Q')
).properties(width=320,height=alt.Step(30),title='Instagram')

IGtext = IGtable2.mark_text(
    align='left',
    baseline='middle',
    dx=3 , 
    color='white'
).encode(
    alt.Text('#Engagement:Q',format=',.4r')
)
IGchart = (IGtable2+IGtext).transform_filter(
    nameselect
)


TWTable2=table2.loc[lambda table2: table2['channel']=='twitter',:]
TWtable2=alt.Chart(TWTable2).transform_fold(
    ['favorite','retweet','share '],as_=['Type','#Engagement']
).mark_bar(size=20,color='#1DA1F2').encode(
    alt.X('#Engagement:Q',axis=None,scale=xscale),
    alt.Y('Type:N',title=None),
    alt.Tooltip('engagement:Q')
).properties(width=350,height=alt.Step(30),title='Twitter')

TWtext = TWtable2.mark_text(
    align='left',
    baseline='middle',
    dx=3,  
    color='white'
).encode(
    alt.Text('#Engagement:Q',format=',.4r')
)
TWchart = (TWtable2+TWtext).transform_filter(
    nameselect
)


YTTable2=table2.loc[lambda table2: table2['channel']=='youtube',:]
YTtable2=alt.Chart(YTTable2).transform_fold(
    ['like','dislike','comment','share '],as_=['Type','#Engagement']
).mark_bar(size=20,color='#FF0000').encode(
    alt.X('#Engagement:Q',axis=None,scale=xscale),
    alt.Y('Type:N',title=None),
    alt.Tooltip('Type:Q')
).properties(width=320,height=110,title='Youtube')

YTtext = YTtable2.mark_text(
    align='left',
    baseline='middle',
    dx=3,  
    color='white'
).encode(
    alt.Text('#Engagement:Q',format=',.4r')
)    
YTchart = (YTtable2+YTtext).transform_filter(
    nameselect
)

a=alt.vconcat(alt.hconcat(FBchart,IGchart),alt.hconcat(TWchart,YTchart))

final3=filter_dropdown|Table3&a


Final3=final3.configure(
    background='#000000'
).configure_axis(
    labelColor='white', labelFont = 'helvetica', labelFontSize=10,
    titleColor='white', titleFont = 'helvetica', titleFontSize=12, titleFontStyle= 'normal',
).configure_view(
    strokeOpacity=0,
    strokeWidth=0
).configure_legend(
    labelColor='white', labelFont = 'helvetica', labelFontSize = 10,
    titleColor='white', titleFont = 'helvetica', titleFontSize = 12, titleFontStyle= 'normal',
    strokeColor=None
).configure_title(
    fontSize=22,
    font='helvetica',
    anchor='start', 
    color='white',
    fontStyle= 'italic',
    frame='group'
).configure_axisX(
    grid=False,
    ticks=False,
    domain=False,
    labelFontSize=12
).configure_axisY(
    grid=False,
    ticks=False,
    domain=False,
    labelFontSize=15
)

Final3
Final3.save('rankbychannelandmonth.html')
Final3.save('rankbychannelandmonth.json')